In [ ]:
#@title Connecting to google drive (Before Running Main Code Connect to Drive)🥇
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive/')

# # Authenticate Google Drive API
from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
auth.authenticate_user() 
creds, _ = default() 
gc = gspread.authorize(creds) 
import gspread_dataframe as gd



import pandas as pd
import shutil
import os  
import datetime
import glob
import re


import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Swiggy

In [ ]:
#@title Main Code

#@title DC code for 2023 files
path2='/content/drive/MyDrive/01_Sales_Reports/B_Swiggy/2023'  # path of files




# ******************************************************************** MAIN CODE *****************************************************************************
_2023_data=pd.DataFrame()
count2=0
for root, dirs, files in os.walk(path2):
  for file in files:
    get_excel1=pd.ExcelFile(f'{root}/{file}')
    count2+=1
    read_payout_invoice1=pd.read_excel(get_excel1,'Payout Invoice',skiprows=range(5))
    read_summary1=pd.read_excel(get_excel1,'Summary',skiprows=range(2))
    read_summary2=pd.read_excel(get_excel1,'All Orders',skiprows=range(1))


    read_payout_invoice1.insert(1,'Year',read_summary2['Order Date'].str.split("-",expand=True)[0],allow_duplicates=False)
    read_payout_invoice1.insert(2,'Payout Period',read_summary1.iloc[13]['Unnamed: 1'],allow_duplicates=False)
    read_payout_invoice1.insert(3,'Res_name',read_summary1.iloc[5]['Unnamed: 1'],allow_duplicates=False)
    read_payout_invoice1.insert(4,'Res_ID',read_summary1.iloc[8]['Unnamed: 1'],allow_duplicates=False)

    read_payout_invoice1.insert(2,'Source_file',file,allow_duplicates=False)
    _2023_data=pd.concat([_2023_data,read_payout_invoice1])
print(count2)


data=_2023_data


#@title droping and renaming colms and nulls rows
data.drop(columns=['Unnamed: 0'],inplace=True)
data.rename(columns={'Unnamed: 1':'Particulars'},inplace=True)


#droping null values
data = data[data['Particulars'].notna()]


# #@title filling null values to 0

data.fillna(0,inplace=True)

#@title Alphabetic Naming of particulars
g={'Number of orders':'A1',
'Item Total':'A2',
'Packaging & Service Charges':'A3',
'Discounts':'A4',
'Net Bill Value':'A5',
'GST on order (including Cess)':'A6',
'Total Customer Payable ':'A7',

'Swiggy Platform Service Fee':'B1',
'Discount on Swiggy Service Fee':'B2',
'Collection Charges':'B3',
'Access Charges':'B4',
'Merchant Cancellation Charges':'B5',
'Call Center Service Fee':'B6',
'Total Swiggy Service Fees (Before Taxes)':'B7',
'Taxes (GST, CESS) over Service Fee':'B8',
'Total Swiggy Service Fees':'B9',

'Cash Pre-Payment to Merchant':'C1',
'Merchant Share Of Cancelled Orders':'C2',
'Delivery Fee Sponsored By Merchant':'C3',
'GST Deduction U/S 9(5)':'C4',
'Refund For Customer Complaints':'C5',
'Order Level Adjustments':'C6',




'High Priority':'D1',
'Outstanding For Previous Weeks':'D2',
'HIGH_PRIORITY':'D3',
'TCS_REFUND':'D4',
'GST_9_5':'D5',
'LONG_DISTANCE_PACK_CHARGES':'D6',
'Refund':'D7',
'CAROUSAL':'D8',
'REFUND_FOR_DISPUTED_ORDER':'D9',
'Refund for Disputed Order':'D10',
'TDS_REFUND':'D11',
'ADS_SALES_REFUND':'D12',
'LAST_WEEK_OUTSTANDING_CURRENT':'D13',
'EXCESS_PAYOUT':'D14',
'LONG_DISTANCE_PACK_CHARGES_REFUND':'D15',
'Other Charges And Refunds':'D16',

'Net Payout (Before TCS) E = A - B - C - D ':'E',
'TCS':'F',
'TDS':'G',
'Net Payout  (E - F - G)':'H'}
data['Naming']=data['Particulars'].map(g)

data['Res_ID']=data['Res_ID'].str.replace('Rest. ID - ',"",regex=True)
# ## ***********************
# os.environ['PYTHONWARNINGS'] = 'always::Warning'
# pd.options.mode.chained_assignment = None
# os.environ['PYTHONWARNINGS'] = '' 
# ## ***********************


data.head(2)

63


,Year,Source_file,Payout Period,Res_name,Res_ID,Particulars,Delivered Orders,Cancelled Orders,Total,Naming
0,2023,invoice Annexure_4780_08032023_1678232074835.xlsx,01 March - 04 March,Dilli Darbar,4780,Number of orders,372.0,7.0,379.0,A1
1,2023,invoice Annexure_4780_08032023_1678232074835.xlsx,01 March - 04 March,Dilli Darbar,4780,Item Total,121200.0,1715.0,122915.0,A2


In [ ]:
#@title go

sales_info=data[data['Particulars'].isin(['Net Bill Value','Merchant Share Of Cancelled Orders','Cash Pre-Payment to Merchant','Merchant Share Of Cancelled Orders','Delivery Fee Sponsored By Merchant','Refund For Customer Complaints'
,'REFUND_FOR_DISPUTED_ORDER','TDS',
"TCS_REFUND"	,
"GST_9_5"	,
"LONG_DISTANCE_PACK_CHARGES_REFUND"	,
"CAROUSAL"	,
"ADS_SALES_REFUND"	,
"TDS_REFUND"	,
"LAST_WEEK_OUTSTANDING_CURRENT"	,
"EXCESS_PAYOUT"	,
"IGCC_REFUND"	,
"OTHER_CHARGES_REIMBURSEMENT_OF_DISCOUNT"	,
"CANCELLATION_SHARING"	])]

sales_info['Particulars']=sales_info['Particulars'].replace("_"," ",regex=True)

replace={"Roj ka Khana(Daily Meals)"	:"Roj Ka Khana",
            "MANN FOOD COURT"	:"Mann Food Court",# 
            "URUVALU BIRYANI"	:"Uruvalu Biryani",#
            "KOOLERZ"	:"Koolerz"}
sales_info['Res_name']=sales_info['Res_name'].replace(replace)

brand_code={"Biryani Hub"	:"HH", #
            "Biryani Box"	:"BB", #
            "Dilli Darbar"	:"DD", #
            "Alpha Biryani"	:"AA",#
            "Roj Ka Khana"	:"RR", # Roj ka Khana
            "Chinese Chaska"	:"CC", #
            "Valluvan Cafe"	:"VV",#
            "Mann Food Court"	:"MM",# Mann Food Court
            "Uruvalu Biryani"	:"UU", # Uruvalu Biryani
            "MFC Juice & Cafe"	:"JJ", #
            "Koolerz"	:"KK",# Koolerz
            "Grill Factory"	:"GG", #
            "Dilli Chaat"	:"TT"} #

sales_info['Brand_code']=sales_info['Res_name'].map(brand_code)

sales_info['New Payout Period']=sales_info['Payout Period'].str.split(" ",expand=True)[1]+sales_info['Payout Period'].str.split(" ",expand=True)[2].str.slice(start=0,stop=3)+sales_info['Year'].astype(str)+"-"+sales_info['Payout Period'].str.split(" ",expand=True)[4]+sales_info['Payout Period'].str.split(" ",expand=True)[5].str.slice(start=0,stop=3)+sales_info['Year'].astype(str)

sales_info['Year'].replace(0,"2023",inplace=True)

invoice_date=sales_info["Payout Period"].str.split(" ",expand=True)[4]+sales_info["Payout Period"].str.split(" ",expand=True)[5].str.slice(start=0,stop=3)+sales_info["Year"].astype(str).str.slice(start=2,stop=4)



sales_info['Particulars'].replace('Net Bill Value','Food Sales',inplace=True)


In [ ]:
#@title go2

l=sales_info["Payout Period"].str.split(" ",expand=True)

sales_info["Invoice Date"]	=	l[4]+" "+l[5].str[:3]+" "+sales_info["Year"].astype(str)
sales_info.loc[:, 'Invoice No.'] ="SW" +sales_info['Brand_code']+l[1]+l[2].str[:3]+l[4]+l[5].str[:3]+sales_info["Year"].astype(str).str[2:4]

sales_info["Customer Name"]	=	'Swiggy'+" "+sales_info['Res_name']
sales_info["Invoice Status"]	=	"Overdue"
sales_info["GST Treatment"]	=	"Consumer"
sales_info["Place of Supply"]	=	"KA"
sales_info["Currency Code"]	=	"INR"
sales_info["Exchange Rate"]	=	"1"
sales_info["Account"]	=	"Sales"
sales_info.rename(columns={'Particulars':"Item Name"},inplace=True)
sales_info["Item Type"]	=	"Goods"
sales_info["Quantity"]	=	"1"
sales_info["Usage unit"]	=	"Count"
sales_info["Is Inclusive Tax"]	=	"Tax"
sales_info["Item Tax"]	=	"GST5"
sales_info["Item Tax Type"]	=	"Tax Group"
sales_info["Item Tax %"]	=	"0%"
sales_info["Supply Type"]	=	"Exempted"
sales_info["Discount Type"]	=	"entity_level"
sales_info["Adjustment"]	=	"Adjustment"
sales_info.rename(columns={'Total':"Item Price"},inplace=True)
sales_info.rename(columns={'invoice':"Invoice No."},inplace=True)

sales_info["E-Commerce Operator Name"]	=	"BUNDL TECHNOLOGIES PRIVATE LIMITED"
sales_info["E-Commerce Operator GSTIN"]	=	"29AAFCB7707D1ZQ"
sales_info["TDS Name"]	=	"Rx-TDS on E-commerce Transactions"
sales_info["TDS Percentage"]	=	"1"
sales_info["TDS Section Code"]	=	"194O"
# sales_info["TDS Amount"]	=	




#condition = (sales_info['Item Name'] != 'Food Sales') & (sales_info['Item Price'] != 0.0) & (sales_info['Item Name'].str.contains('REFUND'),case=False)
condition = (sales_info['Item Name'] != 'Food Sales') & (sales_info['Item Price'] != 0.0) & (sales_info['Item Name'].apply(lambda x: re.match(r'\bREFUND\b', x) is not None))
sales_info.loc[condition, 'Item Price'] = - sales_info.loc[condition, 'Item Price']
sales_info.drop(columns=['Year','Res_name','Res_ID','Delivered Orders', 'Cancelled Orders','Naming', 'Brand_code', 'New Payout Period'],inplace=True)
mask = sales_info['Item Name'] == 'TDS'

condition2 =(sales_info['Item Name'].str.contains('Merchant Share Of Cancelled Orders')) & (sales_info['Item Price'] != 0.0)
sales_info.loc[condition2, 'Item Price'] = - sales_info.loc[condition2, 'Item Price']

condition3 =(sales_info['Item Name'].str.contains('Refund For Customer Complaints')) & (sales_info['Item Price'] != 0.0)
sales_info.loc[condition3, 'Item Price'] = - sales_info.loc[condition3, 'Item Price']


# Assign the 'Item Price' values from rows where 'Item Name' is 'TDS' to the 'TDS Amount' column
sales_info.loc[mask, 'TDS Amount'] = sales_info.loc[mask, 'Item Price']

sales_info = sales_info[::-1]
sales_info['TDS Amount'] = sales_info['TDS Amount'].fillna(method='ffill')
# Reverse the DataFrame back to the original order
sales_info = sales_info[::-1]

sales_info=sales_info[~ (sales_info['Item Name']=='TDS')]


In [ ]:
sales_info.head(2)

,Source_file,Payout Period,Item Name,Item Price,Unnamed: 5,Invoice Date,Invoice No.,Customer Name,Invoice Status,GST Treatment,...,Item Tax %,Supply Type,Discount Type,Adjustment,E-Commerce Operator Name,E-Commerce Operator GSTIN,TDS Name,TDS Percentage,TDS Section Code,TDS Amount
4,invoice Annexure_593472_19042023_1681920227689...,09 April - 15 April,Food Sales,51608.66,0.0,15 Apr 2023,SWGG09Apr15Apr23,Swiggy Grill Factory,Overdue,Consumer,...,0%,Exempted,entity_level,Adjustment,BUNDL TECHNOLOGIES PRIVATE LIMITED,29AAFCB7707D1ZQ,Rx-TDS on E-commerce Transactions,1,194O,516.11
20,invoice Annexure_593472_19042023_1681920227689...,09 April - 15 April,Cash Pre-Payment to Merchant,0.00,0.0,15 Apr 2023,SWGG09Apr15Apr23,Swiggy Grill Factory,Overdue,Consumer,...,0%,Exempted,entity_level,Adjustment,BUNDL TECHNOLOGIES PRIVATE LIMITED,29AAFCB7707D1ZQ,Rx-TDS on E-commerce Transactions,1,194O,516.11


In [ ]:
import gspread_dataframe as gd

wc=gc.open_by_key('13yBVsGXL_p1p2Yi5HXvav8DU7z5jYHWRb3ch-4LCWko')
wx=wc.worksheet('Sheet11')

gd.set_with_dataframe(wx,sales_info) 

In [ ]:
#sales_info.to_excel('/content/drive/MyDrive/MFC1130_Jamshed_Working_Folder/Swig.xlsx',index=False)

<ipython-input-267-6517b775265f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_info['TDS Amount'].fillna(0,inplace=True)


## Zomato

In [ ]:
#@title Main Code

path='/content/drive/MyDrive/01_Sales_Reports/C_Zomato/2023/Apr'  #paste ur path link here




# ********************************************************** MAIN CODE *************************************************************************

count=0
original=pd.DataFrame()


#for root, dirs, files in os.walk(path):
for root , dir,files in  os.walk(path):
  for file in files:
    get_excel=pd.ExcelFile(f'{root}/'+file)
    count+=1
    payout_breakup=pd.read_excel(get_excel,'Payout breakup',skiprows=range(4))
    payout_breakup.insert(0,'Source_files',file,allow_duplicates=False)

    #df2=pd.payout_breakup(get_excel,'Order level',skiprows=range(6))
    summary=pd.read_excel(get_excel,'Summary')

    payout_breakup.insert(3,'Payout_Report_Period',summary.iloc[2]['Unnamed: 2'],allow_duplicates=False)
    payout_breakup.insert(1,'Res_id',summary.iloc[3]['Unnamed: 2'],allow_duplicates=False)
    payout_breakup.rename(columns={'Particular':'Particular' },inplace=True)

    payout_breakup.insert(2,'Res_Name',summary.iloc[6]['Unnamed: 2'],allow_duplicates=False)
    original=pd.concat([original,payout_breakup])
print(count)



data=original

#@title doroping colms
data.drop(columns=['Unnamed: 0','S.No.'],inplace=True)


#@title droping rows from 'Particular' which have nulls

data.dropna(axis='index',how='all',subset=['Particular' ,'Delivered Orders','Cancelled/ Rejected Orders','Total'],inplace=True)


#@title adding Particulars_alpa_serials
Particulars_alpa_serials= {'Number of Orders':'A1', 
       'Subtotal (items total)':'A2',
       'Packaging charge':'A3', 
       'Delivery charge':'A4',
       'Restaurant discount [Promo]':'A5',
       'Restaurant discount [Salt, Freebie, Pro & other discounts]':'A6',
       'Net bill value \n[(1) + (2) + (3) - (4) - (5)]':'A7',

       'Total GST collected from customers':'B1',
       'GST retained by Zomato - Amount of tax paid by Zomato under section 9(5)':'B2',
       'GST to be paid by Restaurant partner to Govt.':'B3',
       'Gross Sales\n[ (A) + (6) ]':'B4',

       'Commissionable amount   [(B) - (6) -(3)]':'D1', 
       'Commission value ':'D2',
       'Payment mechanism fee':'D3', 
       'Logistics charge':'D4',
       'Taxes on Zomato fees':'D5',
       'Tax collected at source + TCS IGST amount':'D6', 
       'TDS 194O amount':'D7',
       'Other Order level deductions':'D8',
        'Customer Compensation/Recoupment':'D9',
       'Delivery Charges Recovery':'D10',
       'Credit note/(Debit Note) adjustment':'D11',
       'Promo recovery adjustment':'D12',
       'Amount received in cash (on self delivery orders)':'D13',
       'Pay later Deductions & other Res id Level Deductions \n(Refer to Annexure : "Addition/ Deduction Details")':'D14',
       'Total Deductions \n[ (10)+(11)+(12)+(13)+(14)+(15)+(16)+(17)+(18) ]':'D15',

       'Cancellation Refund (Auto)':'E16',
       'TDS 194 H and other Res id level Additions\n(Refer to Annexure : "Addition/ Deduction Details")':'E17',
       'Total Additions \n[(19) + (20)]':'E18',

       'Net Payout \n[(B) - (C) + (D) - (7)]':'F19', 
       'Amount Settled':'F20',
       'Pending Amount (Will be credited on next pay-out day)':'F21'}

#Particulars_alpa_serials
data['serial_no']=data['Particular'].map(Particulars_alpa_serials)


data.fillna(0,inplace=True)


## ***********************
os.environ['PYTHONWARNINGS'] = 'always::Warning'
pd.options.mode.chained_assignment = None
os.environ['PYTHONWARNINGS'] = '' 
## ***********************

data.head(2)
# it worked (:

48


,Source_files,Res_id,Res_Name,Payout_Report_Period,Particular,Delivered Orders,Cancelled/ Rejected Orders,Total,serial_no
1,Zomato_Settlement_Report_18387409_10 Apr 2023_...,18387409,Dilli Darbar,10 Apr 2023 - 16 Apr 2023,Number of Orders,802.0,5,807.0,A1
2,Zomato_Settlement_Report_18387409_10 Apr 2023_...,18387409,Dilli Darbar,10 Apr 2023 - 16 Apr 2023,Subtotal (items total),256239.0,0,256239.0,A2


In [ ]:
#@title go

zom_sales=data[data['Particular'].isin(['Net bill value \n[(1) + (2) + (3) - (4) - (5)]','TDS 194O amount','Cancellation Refund (Auto)',"Other Order level deductions",
"Customer Compensation/Recoupment",
"Delivery Charges Recovery",
"Credit note/(Debit Note) adjustment",
"Promo recovery adjustment",
"Extra Inventory Ads (Order Level Deduction)",
"Amount received in cash (on self delivery orders)",
])]

brand_code={"Biryani Hub"	:"HH",
            "Biryani Box"	:"BB",
            "Dilli Darbar"	:"DD",
            "Biryani Box"	:"BB",
            "Alpha Biryani"	:"AA",
            "Roj Ka Khana - Daily Meals"	:"RR",
            "Chinese Chaska"	:"CC",
            "Valluvan Cafe"	:"VV",
            "Mann Food Court"	:"MM",
            "Uruvalu Biryani"	:"UU",
            "MFC Juice & Cafe"	:"JJ",
            "Koolerz"	:"KK",
            "Grill Factory"	:"GG",
            "Dilli Chaat"	:"TT"}

zom_sales['Brand_code']=zom_sales['Res_Name'].map(brand_code)

l1=zom_sales['Payout_Report_Period'].str.split(" ",expand=True)[4]+zom_sales['Payout_Report_Period'].str.split(" ",expand=True)[5]+zom_sales['Payout_Report_Period'].str.split(" ",expand=True)[6].str.slice(start=2,stop=4)
l2=zom_sales['Payout_Report_Period'].str.split(" ",expand=True)[0]+zom_sales['Payout_Report_Period'].str.split(" ",expand=True)[1]
zom_sales['Invoice Number']='ZM'+zom_sales['Brand_code']+l2+l1


zom_sales['Invoice Date']=zom_sales['Payout_Report_Period'].str.split("-",expand=True)[1]



zom_sales['Particular'].replace('Net bill value \n[(1) + (2) + (3) - (4) - (5)]','Food Sales',inplace=True)


In [ ]:
#@title go2

zom_sales["Customer Name"]	=	'Zomato'+" "+zom_sales['Res_Name']
zom_sales["Invoice Status"]	=	"Overdue"
zom_sales["GST Treatment"]	=	"Consumer"
zom_sales["Place of Supply"]	=	"KA"
zom_sales["Currency Code"]	=	"INR"
zom_sales["Exchange Rate"]	=	"1"
zom_sales["Account"]	=	"Sales"
zom_sales.rename(columns={'Particular':"Item Name"},inplace=True)
zom_sales["Item Type"]	=	"Goods"
zom_sales["Quantity"]	=	"1"
zom_sales["Usage unit"]	=	"Count"
zom_sales["Is Inclusive Tax"]	=	"Tax"
zom_sales["Item Tax"]	=	"GST5"
zom_sales["Item Tax Type"]	=	"Tax Group"
zom_sales["Item Tax %"]	=	"0%"
zom_sales["Supply Type"]	=	"Exempted"
zom_sales["Discount Type"]	=	"entity_level"
zom_sales["Adjustment"]	=	"Adjustment"
zom_sales.rename(columns={'Total':"Item Price"},inplace=True)

zom_sales["E-Commerce Operator Name"]	=	"ZOMATO PRIVATE LIMITED"
zom_sales["E-Commerce Operator GSTIN"]	=	"29ABNFM9601R1Z9"
zom_sales["TDS Name"]	=	"Rx-TDS on E-commerce Transactions"
zom_sales["TDS Percentage"]	=	"1"
zom_sales["TDS Section Code"]	=	"194O"
# zom_sales["TDS Amount"]	=	


# condition = (zom_sales['Item Name'] != 'Food Sales') & (zom_sales['Item Price'] != 0.0)
# zom_sales.loc[condition, 'Item Price'] = - zom_sales.loc[condition, 'Item Price']

item_price_negative=["Other Order level deductions","Customer Compensation/Recoupment","Delivery Charges Recovery","Credit note/(Debit Note) adjustment",
                     "Promo recovery adjustment",	"Extra Inventory Ads (Order Level Deduction)",	"Amount received in cash (on self delivery orders)"]

condition=zom_sales['Item Name'].isin(item_price_negative) & (zom_sales['Item Price'] != 0.0)
zom_sales.loc[condition, 'Item Price'] = - zom_sales.loc[condition, 'Item Price']


zom_sales.drop(columns=['Res_Name','Delivered Orders', 'Cancelled/ Rejected Orders', 'Brand_code','serial_no','Brand_code'],inplace=True)
mask = zom_sales['Item Name'] == 'TDS 194O amount'

# Assign the 'Item Price' values from rows where 'Item Name' is 'TDS' to the 'TDS Amount' column
zom_sales.loc[mask, 'TDS Amount'] = zom_sales.loc[mask, 'Item Price']


g={'Food Sales': 1,
 'Other Order level deductions': 2,
 'Customer Compensation/Recoupment': 3,
 'Delivery Charges Recovery': 4,
 'Credit note/(Debit Note) adjustment': 5,
 'Promo recovery adjustment': 6,
 'Extra Inventory Ads (Order Level Deduction)': 7,
 'Amount received in cash (on self delivery orders)': 8,
 'Cancellation Refund (Auto)': 9,
 'TDS 194O amount': 10}

zom_sales['Serail']=zom_sales['Item Name'].map(g)
zom_sales=zom_sales.sort_values('Serail',ascending=True)

# Replace 'NA' values with the corresponding non-NA values based on 'id'
zom_sales['TDS Amount'] = zom_sales.groupby('Source_files')['TDS Amount'].transform(lambda x: x.ffill().bfill())

zom_sales=zom_sales[~ (zom_sales['Item Name']=='TDS 194O amount')]

In [ ]:
zom_sales.head(2)

,Source_files,Res_id,Payout_Report_Period,Item Name,Item Price,Invoice Number,Invoice Date,Customer Name,Invoice Status,GST Treatment,...,Supply Type,Discount Type,Adjustment,E-Commerce Operator Name,E-Commerce Operator GSTIN,TDS Name,TDS Percentage,TDS Section Code,TDS Amount,Serail
7,Zomato_Settlement_Report_18387409_10 Apr 2023_...,18387409,10 Apr 2023 - 16 Apr 2023,Food Sales,231656.24,ZMDD10Apr16Apr23,16 Apr 2023,Zomato Dilli Darbar,Overdue,Consumer,...,Exempted,entity_level,Adjustment,ZOMATO PRIVATE LIMITED,29ABNFM9601R1Z9,Rx-TDS on E-commerce Transactions,1,194O,2316.42,1
7,Zomato_Settlement_Report_19624944_17 Apr 2023_...,19624944,17 Apr 2023 - 23 Apr 2023,Food Sales,41545.40,ZMUU17Apr23Apr23,23 Apr 2023,Zomato Uruvalu Biryani,Overdue,Consumer,...,Exempted,entity_level,Adjustment,ZOMATO PRIVATE LIMITED,29ABNFM9601R1Z9,Rx-TDS on E-commerce Transactions,1,194O,415.49,1


In [ ]:
import gspread_dataframe as gd

wc=gc.open_by_key('13yBVsGXL_p1p2Yi5HXvav8DU7z5jYHWRb3ch-4LCWko')
wx=wc.worksheet('Sheet10')

#gd.set_with_dataframe(wx,zom_sales)

## Thrive

In [ ]:
#@title Main Code

path = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Thrive/Thrive reports/Summary' # copy paste your path





# ***************************************************************************** MAIN CODE **************************************************************************************
count=0
data=pd.DataFrame()

for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            read_csv=pd.read_csv(file_path,names=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'],skiprows=7)

            read_csv2=pd.read_csv(file_path,names=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'])

            read_csv.insert(1,'Period From',read_csv2.iloc[1]['Column2'],allow_duplicates=False)
            read_csv.insert(2,'Period To',read_csv2.iloc[2]['Column2'],allow_duplicates=False)
            read_csv.insert(3,'Res_name',read_csv2.iloc[0]['Column2'],allow_duplicates=False)
            read_csv.insert(0,'Source_file',os.path.basename(file_path),allow_duplicates=False)
            data=pd.concat([data,read_csv])
            count+=1
print('Total Files - ',count)


#@title Droping Null Rows
data.dropna(0,how='all',subset=['Column1',	'Column2'	,	'Column3'	,'Column4', 'Column5','Column6'],inplace=True)


#@title Droping rows and Renaming the cols

data.dropna(0,how='any',subset=['Column1'],inplace=True)

r=	['Column1','Column2'	,	'Column3'	,'Column4', 'Column5','Column6']
d=['Particular','Delivery Orders',	'Take-Away Orders',	'Dine In Orders'	,'Cancelled Orders'	,'Total Orders']
rename_cols=dict(zip(r,d))
#rename_cols
data.rename(columns=rename_cols,inplace=True)


#@title alpha serials for particulars
	
alpha_serials={'Number Of Orders':	'A',
              'Item Sub-Total':	'B',
              'Packaging Or Service Charges':	'C',
              'Delivery Charges'	:'D',
              'Discounts':	'E',
              'Net Bill Value'	:'F',
              'GST Charged to Customer':	'G',
              'Total Customer Payable':	'H',
              'Hashtag Loyalty Service Fees - Direct':	'I',
              'Razorpay Transaction Fee':	'J',
              'Delivery Fee':	'K',
              'Commission - Consumer - New User':'L',
              'Commission - Consumer - Repeat User':'M',
              'Total GST over fees'	:'N',
              'Total Service Fees':	'O',
              'GST Deducted':	'P',
              'TCS Deducted'	:'Q',
              'TDS Deducted':	'R',
              'Net Payable'	:'S'}

#@title add year
data['Year']=data['Period To'].str.split('-',expand=True)[2]


#@title Adding Month and report periods, droping cols


data['Frm day']=data['Period From'].str.split('-',expand=True)[0]
data['Frm Mon']=data['Period From'].str.split('-',expand=True)[1]
data['Frm Mon name']=pd.to_datetime(data['Frm Mon'],format='%m').dt.month_name().str.slice(stop=3)


data['To day']=data['Period To'].str.split('-',expand=True)[0]
data['To Mon']=data['Period To'].str.split('-',expand=True)[1]
data['To Mon name']=pd.to_datetime(data['To Mon'],format='%m').dt.month_name().str.slice(stop=3)



data['New Period From']= data['Frm day'] + " " + data['Frm Mon name'].astype(str)
data['New Period To']= data['To day'] + " " + data['To Mon name'].astype(str)


data['Report Period']=data['New Period From']+"-"+data['New Period To']


data.drop(columns=['Frm Mon','Frm day','To day','To Mon','To Mon name','New Period From','New Period To'],inplace=True)
data.rename(columns={'Frm Mon name':'Month'},inplace=True)


#@title Serial no. fr months
#data['Month'].unique()
months=['Mar','Apr','May','Jun', 'Jul','Aug','Sep','Oct','Nov', 'Dec','Jan']


data.head(2)

Total Files -  4


,Source_file,Particular,Period From,Period To,Res_name,Delivery Orders,Take-Away Orders,Dine In Orders,Cancelled Orders,Total Orders,Year,Month,Report Period
1,summary_Roj Ka Khana_01-04-2023_30-04-2023.csv,Number Of Orders,01-04-2023,30-04-2023,Roj Ka Khana,2,0,0,0,2,2023,Apr,01 Apr-30 Apr
2,summary_Roj Ka Khana_01-04-2023_30-04-2023.csv,Item Sub-Total,01-04-2023,30-04-2023,Roj Ka Khana,446.0,0.0,0.0,0.0,446.0,2023,Apr,01 Apr-30 Apr


In [ ]:
#@title go

thriv_sales=data[data['Particular'].isin(['Net Bill Value','TDS Deducted'])]

brand_code={"Chinese Chaska"	:"CC",
            "Mann Food Court"	:"MM",
            "Dilli Darbar"	:"DD",
            "Roj Ka Khana"	:"RR"}

thriv_sales['Brand_code']=thriv_sales['Res_name'].map(brand_code)

l1=thriv_sales['Report Period'].str.split("-",expand=True)[0].str.replace(" ","",regex=True)
l2=thriv_sales['Report Period'].str.split("-",expand=True)[1].str.replace(" ","",regex=True)
thriv_sales['Invoice Number']='TH'+thriv_sales['Brand_code']+l1+l2+thriv_sales['Year'].str.slice(start=2,stop=4)


thriv_sales['Invoice Date']=thriv_sales['Report Period'].str.split("-",expand=True)[1]+" "+thriv_sales['Year']



thriv_sales['Particular'].replace('Net Bill Value','Food Sales',inplace=True)


In [ ]:
#@title go2

thriv_sales["Customer Name"]	=	'Thrive'+" "+thriv_sales['Res_name']
thriv_sales["Invoice Status"]	=	"Overdue"
thriv_sales["GST Treatment"]	=	"Consumer"
thriv_sales["Place of Supply"]	=	"KA"
thriv_sales["Currency Code"]	=	"INR"
thriv_sales["Exchange Rate"]	=	"1"
thriv_sales["Account"]	=	"Sales"
thriv_sales.rename(columns={'Particular':"Item Name"},inplace=True)
thriv_sales["Item Type"]	=	"Goods"
thriv_sales["Quantity"]	=	"1"
thriv_sales["Usage unit"]	=	"Count"
thriv_sales["Is Inclusive Tax"]	=	"Tax"
thriv_sales["Item Tax"]	=	"GST5"
thriv_sales["Item Tax Type"]	=	"Tax Group"
thriv_sales["Item Tax %"]	=	"0%"
thriv_sales["Supply Type"]	=	"Exempted"
thriv_sales["Discount Type"]	=	"entity_level"
thriv_sales["Adjustment"]	=	"Adjustment"
thriv_sales.rename(columns={'Total Orders':"Item Price"},inplace=True)

thriv_sales["E-Commerce Operator Name"]	=	"Hashtag Loyalty Private Limited"
thriv_sales["E-Commerce Operator GSTIN"]	=	"27AADCH9016M1Z9"
thriv_sales["TDS Name"]	=	"Rx-TDS on E-commerce Transactions"
thriv_sales["TDS Percentage"]	=	"1"
thriv_sales["TDS Section Code"]	=	"194O"


# condition = (thriv_sales['Item Name'] != 'Food Sales') & (thriv_sales['Item Price'] != 0.0)
# thriv_sales.loc[condition, 'Item Price'] = thriv_sales.loc[condition, 'Item Price']
thriv_sales.drop(columns=['Res_name','Delivery Orders', 'Take-Away Orders', 'Dine In Orders','Cancelled Orders','Brand_code'],inplace=True)
mask = thriv_sales['Item Name'] == 'TDS Deducted'

# Assign the 'Item Price' values from rows where 'Item Name' is 'TDS' to the 'TDS Amount' column
thriv_sales.loc[mask, 'TDS Amount'] = thriv_sales.loc[mask, 'Item Price']

thriv_sales=thriv_sales[::-1]
thriv_sales['TDS Amount']=thriv_sales['TDS Amount'].fillna(method='ffill')
thriv_sales=thriv_sales[::-1]


thriv_sales=thriv_sales[~ (thriv_sales['Item Name']=='TDS Deducted')]

In [ ]:
thriv_sales.head(5)

,Source_file,Item Name,Period From,Period To,Item Price,Year,Month,Report Period,Invoice Number,Invoice Date,...,Item Tax %,Supply Type,Discount Type,Adjustment,E-Commerce Operator Name,E-Commerce Operator GSTIN,TDS Name,TDS Percentage,TDS Section Code,TDS Amount
6,summary_Roj Ka Khana_01-04-2023_30-04-2023.csv,Food Sales,01-04-2023,30-04-2023,525.4,2023,Apr,01 Apr-30 Apr,THRR01Apr30Apr23,30 Apr 2023,...,0%,Exempted,entity_level,Adjustment,Hashtag Loyalty Private Limited,27AADCH9016M1Z9,Rx-TDS on E-commerce Transactions,1,194O,5.25
6,summary_Chinese Chaska_01-04-2023_30-04-2023.csv,Food Sales,01-04-2023,30-04-2023,893.0,2023,Apr,01 Apr-30 Apr,THCC01Apr30Apr23,30 Apr 2023,...,0%,Exempted,entity_level,Adjustment,Hashtag Loyalty Private Limited,27AADCH9016M1Z9,Rx-TDS on E-commerce Transactions,1,194O,8.93
6,summary_Dilli Darbar_01-04-2023_30-04-2023.csv,Food Sales,01-04-2023,30-04-2023,18514.0,2023,Apr,01 Apr-30 Apr,THDD01Apr30Apr23,30 Apr 2023,...,0%,Exempted,entity_level,Adjustment,Hashtag Loyalty Private Limited,27AADCH9016M1Z9,Rx-TDS on E-commerce Transactions,1,194O,185.14
6,summary_Mann Food Court_01-04-2023_30-04-2023.csv,Food Sales,01-04-2023,30-04-2023,32231.8,2023,Apr,01 Apr-30 Apr,THMM01Apr30Apr23,30 Apr 2023,...,0%,Exempted,entity_level,Adjustment,Hashtag Loyalty Private Limited,27AADCH9016M1Z9,Rx-TDS on E-commerce Transactions,1,194O,317.46


In [ ]:
import gspread_dataframe as gd

wc=gc.open_by_key('13yBVsGXL_p1p2Yi5HXvav8DU7z5jYHWRb3ch-4LCWko')
wx=wc.worksheet('sheet1')

#gd.set_with_dataframe(wx,thriv_sales)

##Amazon

In [ ]:
# Import the necessary libraries
# import pandas as pd
# from googleapiclient.discovery import build
# from google.oauth2 import service_account

# Define the folder path
folder_path = 'https://drive.google.com/drive/folders/1i2CFkDbCiI_rJ0sGq9p5G2XJdocuMecm'

# Get the folder ID from the provided path
folder_id = folder_path.split('/')[-1]

# Create an empty list to store the file information
file_list = []

# Create a service object using the credentials
service = build('drive', 'v3', credentials=creds)

# Retrieve the list of files in the folder
page_token = None
while True:
    response = service.files().list(q=f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet'",
                                    spaces='drive',
                                    fields='nextPageToken, files(id, name)',
                                    pageToken=page_token).execute()
    file_list.extend(response['files'])
    page_token = response.get('nextPageToken')
    if not page_token:
        break

# Count the number of files read
num_files_read = 0

# Create an empty DataFrame to store the concatenated data
df_concatenated = pd.DataFrame()

# Iterate through each file
for file in file_list:
    # Read the data from the worksheet
    worksheet = gc.open_by_key(file['id']).sheet1
    num_files_read += 1
    values = worksheet.get_all_values()

    # Create a DataFrame from the values and add the "Source File" column
    df = pd.DataFrame(values[1:], columns=values[0])
    df['Source File'] = file['name']

    # Concatenate the DataFrame to the main DataFrame
    df_concatenated = pd.concat([df_concatenated, df])

# Reset the index of the concatenated DataFrame
df_concatenated = df_concatenated.reset_index(drop=True)

# Display the number of files read
print("Number of files read:", num_files_read)


Number of files read: 39


In [ ]:
#@title go
# Provide the folder path (update this with the correct path to your folder)
folder_path = 'https://drive.google.com/drive/folders/1i2CFkDbCiI_rJ0sGq9p5G2XJdocuMecm'

# Get the folder ID from the provided path
folder_id = folder_path.split('/')[-1]
df_concatenated = pd.DataFrame()

service = build('drive', 'v3', credentials=creds)

file_list = []
page_token = None

while True:
 response = service.files().list(q=f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet'",
 spaces='drive',
 fields='nextPageToken, files(id, name)',
 pageToken=page_token).execute()
 file_list.extend(response['files'])
 page_token = response.get('nextPageToken')
 if not page_token:
  break

# Count the number of files read
num_files_read = 0

# Iterate through each file
for file in file_list:
 worksheet = gc.open_by_key(file['id']).sheet1
 num_files_read += 1
 values = worksheet.get_all_values()

 df = pd.DataFrame(values[1:], columns=values[0])
 file_name = next((item['name'] for item in file_list if item['id'] == file['id']), None)
 df_concatenated = pd.concat([df_concatenated, df])

 

df_concatenated = df_concatenated.reset_index(drop=True)


# Display the number of files read
print("Number of files read:", num_files_read) add source file column 


Number of files read: 39


In [ ]:
df_concatenated.head(2)

,Order Id#,Restaurant Outlet Name,Restaurant Merchant Id,Delivery Type,Local Purchase Date,Local Purchase Time,Delivery Pincode,Item ASIN,Item Name,Sub Item,...,TDS Refund,GST paid by Amazon Refund,Amazon Cost of Refund,Total Refund by Amazon,Total Refund by Merchant,Net Payable to Merchant post Refund,Source File,Month,Year,Month_name
0,D01-9959192-8904627,Uruvalu Biryani,200249,AmazonDelivery,25-12-2022,20:21:44,560066,B0B31T6P4W,Tandoori Chicken (Best Seller),No,...,0,0,0,0,0,79.68,200248-URUVALU-ITEM_LEVEL-01APR2022-31MAR2023,12,2022,Dec
1,D01-9959192-8904627,Uruvalu Biryani,200249,AmazonDelivery,25-12-2022,20:21:44,560066,B0BH146XZ2,Quarter,Yes,...,0,0,0,0,0,0,200248-URUVALU-ITEM_LEVEL-01APR2022-31MAR2023,12,2022,Dec


In [ ]:
# Define the month-to-serial number mapping dictionary

month_mapping = {"Jan":'01','Feb':'02','Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}
reversed_month_mapping = {v: k for k, v in month_mapping.items()}
reversed_month_mapping

In [ ]:
df_concatenated['Month']=df_concatenated['Local Purchase Date'].astype(str).str.split("-",expand=True)

In [ ]:
df_concatenated['Year']=df_concatenated['Local Purchase Date'].astype(str).str.split("-",expand=True)[2]
df_concatenated['Month_name']=df_concatenated['Month'].map(reversed_month_mapping)

In [ ]:
df_concatenated['Net Payable to Merchant post Refund']=df_concatenated['Net Payable to Merchant post Refund'].astype(float)

# df_concatenated['Local Purchase Date']=pd.to_datetime(df_concatenated['Local Purchase Date'])
# df_concatenated['Month']=df_concatenated['Local Purchase Date'].dt.month_name() 
# df_concatenated['Year']=df_concatenated['Local Purchase Date'].dt.year

In [ ]:
columns=["Item Unit Price",
        "Net Sales Value",
        "GST to be paid by Merchant",
        "GST to be paid by Amazon",
        "Total GST",
        "Customer Payable",
        "Amazon Seller Fee without GST",
        "CGST on Amazon Seller Fee",
        "SGST on Amazon Seller Fee",
        "IGST on Amazon Seller Fee",
        "Total Seller Fee including GST",
        "Merchant Share Net of Amazon Fee",
        "TCS CGST",
        "TCS SGST",
        "TCS IGST",
        "TDS",
        "Merchant Payable Amount post TCS and TDS Deduction",
        "Amazon Seller Fee without GST Refunded",
        "Seller Fee GST Refunded",
        "Total Fee Refunded",
        "TCS Refund",
        "TDS Refund",
        "GST paid by Amazon Refund",
        "Amazon Cost of Refund",
        "Total Refund by Amazon",
        "Total Refund by Merchant",
        "Net Payable to Merchant post Refund"]

df_concatenated[columns] = df_concatenated[columns].replace('', 0).astype(float)


In [ ]:
df_concatenated['Month_Year']=df_concatenated['Month'].str.slice(start=0,stop=3)+"-"+df_concatenated['Year'].astype(str)

In [ ]:
df_concatenated.head(2)

,Source_file,Order Id#,Restaurant Outlet Name,Restaurant Merchant Id,Delivery Type,Local Purchase Date,Local Purchase Time,Delivery Pincode,Item ASIN,Item Name,...,TCS Refund,TDS Refund,GST paid by Amazon Refund,Amazon Cost of Refund,Total Refund by Amazon,Total Refund by Merchant,Net Payable to Merchant post Refund,Month,Year,Month_Year
0,200248-URUVALU-ITEM_LEVEL-01APR2022-31MAR2023,D01-9959192-8904627,Uruvalu Biryani,200249,AmazonDelivery,2022-12-25,20:21:44,560066,B0B31T6P4W,Tandoori Chicken (Best Seller),...,0.0,0.0,0.0,0.0,0.0,0.0,79.68,December,2022,Dec-2022
1,200248-URUVALU-ITEM_LEVEL-01APR2022-31MAR2023,D01-9959192-8904627,Uruvalu Biryani,200249,AmazonDelivery,2022-12-25,20:21:44,560066,B0BH146XZ2,Quarter,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,December,2022,Dec-2022


In [ ]:
dne=df_concatenated.pivot_table(values=columns,index=['Source_file','Month_Year','Restaurant Outlet Name'],aggfunc='sum')

In [ ]:
dne=pd.DataFrame(dne)

In [ ]:
dne=dne.reset_index()
dne.head(2)

,Source_file,Month_Year,Restaurant Outlet Name,Amazon Cost of Refund,Amazon Seller Fee without GST,Amazon Seller Fee without GST Refunded,CGST on Amazon Seller Fee,Customer Payable,GST paid by Amazon Refund,GST to be paid by Amazon,...,TCS IGST,TCS Refund,TCS SGST,TDS,TDS Refund,Total Fee Refunded,Total GST,Total Refund by Amazon,Total Refund by Merchant,Total Seller Fee including GST
0,200248-URUVALU-ITEM_LEVEL-01APR2021-31MAR2022,Apr-2022,Uruvalu Biryani,0.00,77.41,0.00,6.98,812.73,0.0,0.0,...,0.0,0.0,0.0,7.74,0.0,0.00,38.73,0.00,0.00,91.37
1,200248-URUVALU-ITEM_LEVEL-01APR2021-31MAR2022,Aug-2022,Uruvalu Biryani,396.12,219.89,56.49,19.79,2308.98,0.0,0.0,...,0.0,0.0,0.0,21.98,0.0,66.65,109.98,469.77,123.48,259.47


In [ ]:
df_concatenated.shape

(70974, 57)

In [ ]:
import gspread_dataframe as gd

wc=gc.open_by_key('1cmlXGuq1V0C7VbrqRHSWu-lQrnrnOkRMak-XyrfBidM')
wx=wc.worksheet('Sheet5')

gd.set_with_dataframe(wx,df_concatenated)

## Ingnore

In [ ]:
# sales_info['Particulars'].replace('Net Bill Value','Food Sales',inplace=True)
# pivot=sales_info.pivot_table(columns=['Naming','Particulars'],values='Total',index=['Source_file','invoice'],aggfunc='sum')
# pivot.head(2)

# pivot_db[pivot_db['Source_file']=='invoice Annexure_180796_04052023_1683219337889.xlsx']

# pivot_db=pd.DataFrame(pivot)
# pivot_db=pivot_db.reset_index()
# pivot_db.columns=pivot_db.columns.droplevel(0)
# pivot_db.columns.values[0]='Source_file'
# pivot_db.columns.values[1]='Invoice'
# pivot_db.fillna(0,inplace=True)
# pivot_db.head(2)

In [ ]:
#pivot.to_excel('/content/drive/MyDrive/MFC1130_Jamshed_Working_Folder/Swiggy_Sales.xlsx',index=False)